In [1]:
import pandas as pd
import numpy as np
import random
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, plot_confusion_matrix, make_scorer
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from matplotlib import pyplot as plt

In [5]:
#Loading the dataset
dataset_path = "dataset1_sydney.csv"
sydney_dataset = pd.read_csv(dataset_path)
sydney_dataset = sydney_dataset.sample(sydney_dataset.shape[0])

header = sydney_train.columns

In [6]:
def unique_vals(rows, col):
    """Find the unique values for a column in a dataset."""
    return set([row[col] for row in rows])

#######
# Demo:
# unique_vals(training_data, 0)
# unique_vals(training_data, 1)
#######

In [7]:
def class_counts(rows):
    """Counts the number of each type of example in a dataset."""
    counts = {}  # a dictionary of label -> count.
    for row in rows:
        # in our dataset format, the label is always the last column
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

#######
# Demo:
# class_counts(training_data)
#######

In [8]:
def is_numeric(value):
    """Test if a value is numeric."""
    return isinstance(value, int) or isinstance(value, float)

#######
# Demo:
# print(is_numeric(7), is_numeric("Red"))
# 
#######

In [9]:
class Question:
    """A Question is used to partition a dataset.
    This class just records a 'column number' (e.g., 0 for Color) and a
    'column value' (e.g., Green). The 'match' method is used to compare
    the feature value in an example to the feature value stored in the
    question. See the demo below.
    """

    def __init__(self, column, value):
        self.column = column 
        self.value = value 
    def match(self, example):
        # Compare the feature value in an example to the
        # feature value in this question.
        val = example[self.column] #example[1]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
        condition = "=="
        if is_numeric(self.value):
            condition = ">"
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))
#######
# Demo:
# Let's write a question for a numeric attribute
# Question(1, 3)
# # How about one for a categorical attribute
# q = Question(0, 'Green')
# # Let's pick an example from the training set...
# example = training_data[0]
# # ... and see if it matches the question
# q.match(example)
# q.__repr__()
#######

In [10]:
def partition(rows, question):
    """Partitions a dataset.
    For each row in the dataset, check if it matches the question. If
    so, add it to 'true rows', otherwise, add it to 'false rows'.
    """
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [11]:
def gini(rows):
    """Calculate the Gini Impurity for a list of rows.
    There are a few different ways to do this, I thought this one was
    the most concise. See:
    https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity
    """
    counts = class_counts(rows)
    impurity = 1
    for label in counts:
        prob_of_label = counts[label] / float(len(rows))
        impurity -= prob_of_label**2
    return impurity


In [12]:
def info_gain(left, right, current_uncertainty):
    """Information Gain.
    The uncertainty of the starting node, minus the weighted impurity of
    two child nodes.
    """
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [13]:
def find_best_split(rows, random_subspace):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    current_uncertainty = gini(rows)
    column_indices = list(range(len(rows[0]) - 1))  # number of columns
    
    if random_subspace and random_subspace <= len(column_indices):
        column_indices = random.sample(population = column_indices, k = random_subspace)
    
    for col in column_indices:  # for each feature

        values = set([row[col] for row in rows])  # unique values in the column

        for val in values:  # for each value

            question = Question(col, val)

            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the
            # dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            # You actually can use '>' instead of '>=' here
            # but I wanted the tree to look a certain way for our
            # toy dataset.
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [15]:
class Leaf:
    """A Leaf node classifies data.
    This holds a dictionary of class (e.g., "Apple") -> number of times
    it appears in the rows from the training data that reach this leaf.
    """

    def __init__(self, rows):
        self.predictions = class_counts(rows)


In [16]:
class Decision_Node:
    """A Decision Node asks a question.
    This holds a reference to the question, and to the two child nodes.
    """

    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [17]:
def build_tree(rows, random_subspace):
    """Builds the tree.
    Rules of recursion: 1) Believe that it works. 2) Start by checking
    for the base case (no further information gain). 3) Prepare for
    giant stack traces.
    """

    # Try partitioing the dataset on each of the unique attribute,
    # calculate the information gain,
    # and return the question that produces the highest gain.
    gain, question = find_best_split(rows,random_subspace)
    # Base case: no further info gain
    # Since we can ask no further questions,
    # we'll return a leaf.
    if gain == 0:
        return Leaf(rows)

    # If we reach here, we have found a useful feature / value
    # to partition on.
    true_rows, false_rows = partition(rows, question)

    # Recursively build the true branch.
    true_branch = build_tree(true_rows, random_subspace)

    # Recursively build the false branch.
    false_branch = build_tree(false_rows, random_subspace)

    # Return a Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    # dependingo on the answer.
    return Decision_Node(question, true_branch, false_branch)

In [18]:
def print_tree(node, spacing=""):
    """World's most elegant tree printing function."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [19]:
def classify(row, node):
    """See the 'rules of recursion' above."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions
   
    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node,
    # to the example we're considering.
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)


In [20]:
def print_leaf(counts):
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
        
    return probs


In [21]:
def bootstrapping(dataset, num_bootstrap):
    """
    Idea is to create random sample for each decision tree that we create
    """
    bootstrap_indices = np.random.randint(low=0, high=len(dataset), size= num_bootstrap)
    df_bootstrapped = dataset.iloc[bootstrap_indices]
    return df_bootstrapped

In [22]:
def random_forest(dataset, n_features,n_decision_trees = 50, num_bootstrap=50):
    forest = []
    for i in range(n_decision_trees):
        bootstrapped_data = bootstrapping(dataset, num_bootstrap)
        bootstrapped_data = bootstrapped_data.to_numpy()
        tree = build_tree(bootstrapped_data, random_subspace=n_features)
        forest.append(tree)
    return forest

In [ ]:
# forest = random_forest(sydney_train, n_features = 10, n_decision_trees = 10, num_bootstrap = 500)

In [23]:
def random_forest_predictions(test_data, forest):
    test_predictions = {}
    test_data = test_data.to_numpy()
    
    for i in range(len(forest)):
        column_name = f"tree_{i}"
        predicted= []
        actual = []
        
        for row in test_data:
            most_probable = print_leaf(classify(row, forest[i]))
            predicted += [next(iter(most_probable))]
            actual += [row[-1]]
        test_predictions[column_name] = predicted

        
    test_predictions = pd.DataFrame(test_predictions)
    print(test_predictions)
    random_forest_predictions = test_predictions.mode(axis=1)[0]
    
    return random_forest_predictions, actual

In [25]:
def get_overall_cm(df):
    classes = df['target'].nunique()
    if classes < 2:
        classes = 2
    overall_cm = []
    for i in range(classes):
        overall_cm.append( [0] * classes)
    return overall_cm

In [33]:
from sklearn.metrics import confusion_matrix

def Kfold(df,n_folds, test_size, overall_cm):
    # Iterate through all folds
    print("THIS IS I, HELLO", n_folds)
    for i in range(n_folds):
        #split test and train
        test = df.iloc[test_size * i : test_size*i+test_size]
        
        train = df.iloc[test_size:]

        forest = random_forest(train, n_features = 10, n_decision_trees = 10, num_bootstrap = 500)

        predicted, actual = random_forest_predictions(test, forest)
        print(classification_report(predicted, actual))
        # generate and append cm 
        overall_cm = overall_cm + confusion_matrix(actual, predicted)
    return overall_cm

if __name__ == "__main__":
    # KFOLD take in n folds and the size of dataset
    
    # DATASET1 SYDNEY
    n_folds = 2
    test_size = int(sydney_dataset.shape[0]/n_folds)

    # get overall cm returns a Confusion matrix with 0's shape depends on (classes)
    print(Kfold(sydney_dataset,n_folds, test_size, get_overall_cm(sydney_dataset)))

THIS IS I, HELLO 2
         tree_0      tree_1      tree_2  ...      tree_7      tree_8      tree_9
0        spiral      spiral      spiral  ...      spiral      spiral      spiral
1    elliptical  elliptical  elliptical  ...  elliptical  elliptical  elliptical
2    elliptical  elliptical  elliptical  ...  elliptical  elliptical  elliptical
3        spiral      spiral      spiral  ...      spiral  elliptical      spiral
4    elliptical  elliptical  elliptical  ...  elliptical  elliptical  elliptical
..          ...         ...         ...  ...         ...         ...         ...
255  elliptical  elliptical  elliptical  ...  elliptical  elliptical  elliptical
256      spiral      spiral      spiral  ...      spiral      spiral      spiral
257      spiral      spiral      spiral  ...      spiral      spiral      spiral
258  elliptical  elliptical  elliptical  ...  elliptical  elliptical  elliptical
259      spiral      spiral      spiral  ...      spiral      spiral      spiral

[260 row